## This notebook focus on generating ethnic specific AF for the HC matrix table

In [1]:
%%configure -f
{"driverMemory": "6000M"}

In [2]:
import hail as hl
hl.init(sc)

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2-amzn-0
SparkUI available at 
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.80-4ccfae1ff293
LOGGING: writing to 

In [3]:
# Read the matrix table
release14_hc_mt_uri =   "SG10K-SV-Release-1.4-HighConfidenceSV.mt"
release14_sample_metadata_uri = "2021_06_18_supplier_metadata.n10714_replacespace.txt"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Read the matrix table
mt = hl.read_matrix_table(release14_hc_mt_uri)
print("Samples: %d; Variants: %d; Entries: %d" % (mt.count_cols(), mt.count_rows(), mt.entries().count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Samples: 5487; Variants: 73035; Entries: 400743045
2024-04-17 02:46:49 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'

In [5]:
### import metadata
t = hl.import_table(release14_sample_metadata_uri, delimiter="\t", quote="\"", impute=True)
#t.describe()

## annotate the hail mt with metadata
t = t.rename({"NPM_Research_ID" : "s"})
t = t.key_by(t.s)
mt = mt.annotate_cols(metadata = t[mt.s])
mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'metadata': struct {
        Multiplex_Pool_ID: str, 
        Supplier_ID: str, 
        GIS_Internal_Sample_ID: str, 
        Site_Supplying_Sample: str, 
        Year_Of_Birth: int32, 
        Supplied_Gender: str, 
        Self_Reported_Ethnicity: str, 
        Extraction_Kit: str, 
        Date_Of_DNA_Extraction: str, 
        Plate_Position: str, 
        Plate_Name: str, 
        Version_Of_Consent_Form_Signed: str, 
        Sequencing_Depth: str, 
        NPM_Research_ID_Created_By_Username: str, 
        NPM_Research_ID_Creation_Date: str, 
        Comments_Entered_When_NPM_Research_ID_Created: str, 
        Description_Entered_When_NPM_Research_ID_Created: str, 
        ELM_Project_ID: str, 
        ELM_Project_Title: str, 
        ELM_Project_PI: str, 
        Species_Of_Sample_Sequenced: str, 
        Tehcnique_For_Sequencing: str, 
      

In [6]:

# ancestry_call_stats_mt = (mt.group_cols_by(ancestry = mt.metadata.Self_Reported_Ethnicity)
#                             .aggregate(call_stats = hl.agg.call_stats(mt.GT, mt.alleles))
# would have been nice but manipulating the resulting mt ( row=ancestry; col=locus entry=call_stats(struct) )
# flattening it and haing ancestry filed values becoming parts of filed key name is a bit tricky
# this is however how gnomad does it to create pop freq for their exsport to ES (variant browser) See `def generate_frequency_data()` 
# instead we'll use hl.agg.call_stats() on filter_cols( Self_Reported_Ethnicity == ${ancestry}) for each of the 3 ancestries
# ...creating info.AC|AN|AF_${ancesrty} with ${ancestry}) == "Chinese" 
_mt = mt.filter_cols( (mt.metadata.Self_Reported_Ethnicity == "Chinese") , keep=True)
_call_stats = _mt.annotate_rows(call_stats = hl.agg.call_stats(_mt.GT, _mt.alleles)).rows().select('call_stats')
_call_stats.describe()
mt = mt.annotate_rows( info = mt.info.annotate(
     AC_CHINESE = _call_stats[mt.row_key].call_stats.AC[1],
     AN_CHINESE = _call_stats[mt.row_key].call_stats.AN,
     AF_CHINESE = _call_stats[mt.row_key].call_stats.AF[1],
))
# ...creating info.AC|AN|AF_${ancesrty} with ${ancestry}) == "Malay" 
_mt = mt.filter_cols( (mt.metadata.Self_Reported_Ethnicity == "Malay") , keep=True)
_call_stat = _mt.annotate_rows(call_stats = hl.agg.call_stats(_mt.GT, _mt.alleles)).rows().select('call_stats')
mt = mt.annotate_rows( info = mt.info.annotate(
    AC_MALAY = _call_stat[mt.row_key].call_stats.AC[1],
    AN_MALAY = _call_stat[mt.row_key].call_stats.AN,
    AF_MALAY = _call_stat[mt.row_key].call_stats.AF[1],
))
# ...creating info.AC|AN|AF_${ancesrty} with ${ancestry}) == "Indian" 
_mt = mt.filter_cols( (mt.metadata.Self_Reported_Ethnicity == "Indian") , keep=True)
_call_stat = _mt.annotate_rows(call_stats = hl.agg.call_stats(_mt.GT, _mt.alleles)).rows().select('call_stats')
mt = mt.annotate_rows( info = mt.info.annotate(
    AC_INDIAN = _call_stat[mt.row_key].call_stats.AC[1],
    AN_INDIAN = _call_stat[mt.row_key].call_stats.AN,
    AF_INDIAN = _call_stat[mt.row_key].call_stats.AF[1],
))

mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'call_stats': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        homozygote_count: array<int32>
    } 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'metadata': struct {
        Multiplex_Pool_ID: str, 
        Supplier_ID: str, 
        GIS_Internal_Sample_ID: str, 
        Site_Supplying_Sample: str, 
        Year_Of_Birth: int32, 
        Supplied_Gender: str, 
        Self_Reported_Ethnicity: str, 
        Extraction_Kit: str, 
        Date_Of_DNA_Extraction: str, 
        Plate_Position: str, 
        Plate_Name: str, 
        Version_Of_Consent_Form_Signed: str, 
     

In [7]:
## write the resulting mt
release14_hc_meta_mt_uri =   "SG10K-SV-Release-1.4-HighConfidenceSV-WithMetadata.mt"
mt.write(release14_hc_meta_mt_uri, overwrite=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-04-17 02:48:16 Hail: INFO: wrote matrix table with 73035 rows and 5487 columns in 91 partitions to SG10K-SV-Release-1.4-HighConfidenceSV-WithMetadata.mt